In [1]:
from detection.detection import anonymize_on_video, cascade_detect_image
from existing.viola.viola import viola_anonymize_on_video, viola_detect_on_img
from existing.mtcnn.mtcnn import mtcnn_anonymize_on_video, mtcnn_detect_on_img
from existing.ssd.ssd import ssd_detect_on_video, ssd_detect_on_img

# Detection on image

Viola

In [4]:
file1 = 'WIDER_train\images/6--Funeral/6_Funeral_Funeral_6_14.jpg'
file2 = 'WIDER_train\images/35--Basketball/35_Basketball_Basketball_35_98.jpg'
test_files = [file1, file2]

for i in range(2):
    viola_detect_on_img(test_files[i], True, f"test_images/test_viola{i+1}.jpg")

In [ ]:
for i in range(2):
    mtcnn_detect_on_img(test_files[i], True, f"test_images/test_mtcnn{i+1}.jpg")

In [6]:
for i in range(2):
    ssd_detect_on_img(test_files[i], True, f"test_images/test_ssd{i+1}.jpg")

In [ ]:
for i in range(2):
    cascade_detect_image(test_files[i], True, f"test_images/test_cascade{i+1}.jpg")

In [2]:
import random
import os
import cv2

# Define the path to the validation set and annotations
val_path = 'WIDER_val'
annotations_path = 'wider_face_split/wider_face_val_bbx_gt.txt'

# Create a list of file paths for the images and annotations
image_paths = []
annotation_boxes = []
current_image_path = ""
with open(annotations_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.endswith('.jpg'):
            current_image_path = os.path.join(val_path, 'images', line)
            image_paths.append(current_image_path)
        else:
            if line != "":
                num_of_bbx = int(line)
                current_annotation_boxes = []
                for i in range(num_of_bbx):
                    next_line = file.readline()
                    x1, y1, w, h, _, _, _, _, _, _ = list(map(int, next_line.strip().split()))
                    current_annotation_boxes.append((x1, y1, x1+w, y1+h))
                annotation_boxes.append(current_annotation_boxes)

random_indices = random.sample(range(len(image_paths)), 100)
sample_image_paths = [image_paths[i] for i in random_indices]
sample_annotation_boxes = [annotation_boxes[i] for i in random_indices]

In [24]:
# Load the Viola-Jones cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define some variables to keep track of the counts
total_faces = 0
detected_faces = 0
false_negatives = 0
false_positives = 0

# Iterate over all the images and annotations
for i in range(len(sample_image_paths)):
    # Read the image
    image = cv2.imread(sample_image_paths[i])
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get the bounding boxes for all faces in the image
    total_faces += len(sample_annotation_boxes[i])

    # Detect faces in the image using the classifier
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    detected_faces += len(faces)

        # Compare the detections with the annotation boxes
    for (x, y, w, h) in faces:
        matched = False
        for (xmin, ymin, xmax, ymax) in sample_annotation_boxes[i]:
            if (xmin <= x <= xmax) and (ymin <= y <= ymax):
                matched = True
                break
        if not matched:
            false_positives += 1

    # Count false negatives
    for (xmin, ymin, xmax, ymax) in sample_annotation_boxes[i]:
        matched = False
        for (x, y, w, h) in faces:
            if (xmin <= x <= xmax) and (ymin <= y <= ymax):
                matched = True
                break
        if not matched:
            false_negatives += 1

# Compute the false negative ratio
false_negative_ratio = false_negatives / total_faces
print('False negative ratio for Viola-Jones:', false_negative_ratio)

# Compute the false positive ratio
false_positive_ratio = false_positives / total_faces
print('False positive ratio for Viola-Jones:', false_positive_ratio)

False negative ratio for Viola-Jones: 0.9931740614334471
False positive ratio for Viola-Jones: 0.07713310580204778


In [4]:
from model.model import predict
from tensorflow import keras
model = keras.models.load_model('best_model.h5', compile=False)

def detect(frame, cascade, model, single_shot=False):
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.equalizeHist(frame_gray)
    faces = cascade.detectMultiScale(frame_gray, 1.05, 3)
    for (x, y, w, h) in faces:
        frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)
        result = predict(model, frame_gray[y:y+h, x:x+w])
        if result == 1:
            frame = cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
    cv2.imshow('Capture - Face detection', frame)
    if single_shot:
        cv2.waitKey(0)

# Load the Viola-Jones cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define some variables to keep track of the counts
total_faces = 0
detected_faces = 0
cnn_approved_faces = 0
false_negatives = 0
false_positives = 0

# Iterate over all the images and annotations
for i in range(len(sample_image_paths)):
    # Read the image
    image = cv2.imread(sample_image_paths[i])
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get the bounding boxes for all faces in the image
    total_faces += len(sample_annotation_boxes[i])

    # Detect faces in the image using the classifier
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    detected_faces += len(faces)
    cnn_faces = []

    for (x, y, w, h) in faces:
        result = predict(model, gray[y:y+h, x:x+w])
        if result == 1:
            cnn_approved_faces += 1
            cnn_faces.append((x, y, w, h))

        # Compare the detections with the annotation boxes
    for (x, y, w, h) in cnn_faces:
        matched = False
        for (xmin, ymin, xmax, ymax) in sample_annotation_boxes[i]:
            if (xmin <= x <= xmax) and (ymin <= y <= ymax):
                matched = True
                break
        if not matched:
            false_positives += 1

    # Count false negatives
    for (xmin, ymin, xmax, ymax) in sample_annotation_boxes[i]:
        matched = False
        for (x, y, w, h) in cnn_faces:
            if (xmin <= x <= xmax) and (ymin <= y <= ymax):
                matched = True
                break
        if not matched:
            false_negatives += 1

# Compute the false negative ratio
false_negative_ratio = false_negatives / total_faces
print('False negative ratio for Viola-CNN Cascade:', false_negative_ratio)

# Compute the false positive ratio
false_positive_ratio = false_positives / total_faces
print('False positive ratio for Viola-CNN Cascade:', false_positive_ratio)

1/1 [==============================] - 0s 24ms/step
False negative ratio for Viola-CNN Cascade: 0.9847715736040609
False positive ratio for Viola-CNN Cascade: 0.09354604786076867


In [5]:
model = keras.models.load_model('best_model.h5', compile=False)
from model.model import load_dataset

train, val, test = load_dataset()

1.jpg
1
10.jpg
2
100.jpg
3
1000.jpg
4
10000.jpg
5
100000.jpg
6
100001.jpg
7
100002.jpg
8
100003.jpg
9
100005.jpg
11
100006.jpg
12
100007.jpg
13
100008.jpg
14
100009.jpg
15
10001.jpg
16
100010.jpg
17
100011.jpg
18
100012.jpg
19
100014.jpg
21
100015.jpg
22
100016.jpg
23
100017.jpg
24
100018.jpg
25
100019.jpg
26
10002.jpg
27
100020.jpg
28
100021.jpg
29
100023.jpg
31
100024.jpg
32
100025.jpg
33
100026.jpg
34
100027.jpg
35
100028.jpg
36
100029.jpg
37
10003.jpg
38
100030.jpg
39
100032.jpg
41
100033.jpg
42
100034.jpg
43
100035.jpg
44
100036.jpg
45
100037.jpg
46
100038.jpg
47
100039.jpg
48
10004.jpg
49
100041.jpg
51
100042.jpg
52
100043.jpg
53
100044.jpg
54
100045.jpg
55
100046.jpg
56
100047.jpg
57
100048.jpg
58
100049.jpg
59
100050.jpg
61
100051.jpg
62
100052.jpg
63
100053.jpg
64
100054.jpg
65
100055.jpg
66
100056.jpg
67
100057.jpg
68
100058.jpg
69
10006.jpg
71
100060.jpg
72
100061.jpg
73
100062.jpg
74
100063.jpg
75
100064.jpg
76
100065.jpg
77
100066.jpg
78
100067.jpg
79
100069.jpg
81
10007.j

KeyboardInterrupt: 

In [3]:
from model.model import predict
from tensorflow import keras
import cv2
model = keras.models.load_model('best_model.h5', compile=False)
# Load the Viola-Jones cascade classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Define some variables to keep track of the counts
total_faces = 0
detected_faces = 0
cnn_approved_faces = 0
false_negatives = 0
false_positives = 0

# Iterate over all the images and annotations
for i in range(len(sample_image_paths)):
    # Read the image
    image = cv2.imread(sample_image_paths[i])
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Get the bounding boxes for all faces in the image
    total_faces += len(sample_annotation_boxes[i])

    # Detect faces in the image using the classifier
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    detected_faces += len(faces)

    for (x, y, w, h) in faces:
        result = predict(model, gray[y:y+h, x:x+w])
        if result == 1:
            cnn_approved_faces += 1

1/1 [==============================] - 0s 40ms/step
150
152


In [ ]:
from model.model import main as model_eval
